In [10]:
import pandas as pd
import json
from datetime import datetime, timedelta
import random

In [11]:
csv_file_path = 'TabelaNutrIA-Tratada.csv'
df = pd.read_csv(csv_file_path, sep=',')
jsonl_file_path = 'output-backend.jsonl'

In [12]:
dates = {
    'hoje': datetime.now().strftime('%Y-%m-%d'),
    'ontem': (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d'),
    'amanha': (datetime.now() + timedelta(days=1)).strftime('%Y-%m-%d')
}

datas_chaves = list(dates.keys())
valores_porcao_padrao = [1, 50, 100, 200]
valores_quantidades = [1, 25, 30, 40, 50, 55, 75, 60, 93, 100]


In [13]:
content = f"Você é um assistente nutricional especializado em invocar funções para ajudar o usuário a preencher o seu registro de alimentos. Se atente a função solicitada e seu respectivo schema!"

refeicoes_usuario = {"1":{"numero_refeicao":1,"nome_refeicao":"Café","totalProteina":0,"totalCarboidrato":0,"totalGordura":0,"totalAlcool":0,"totalKcal":0},"2":{"numero_refeicao":2,"nome_refeicao":"Almoço","totalProteina":0,"totalCarboidrato":0,"totalGordura":0,"totalAlcool":0,"totalKcal":0},"3":{"numero_refeicao":3,"nome_refeicao":"Jantar","totalProteina":0,"totalCarboidrato":0,"totalGordura":0,"totalAlcool":0,"totalKcal":0},"4":{"numero_refeicao":4,"nome_refeicao":"Lanche Da Tarde","totalProteina":0,"totalCarboidrato":0,"totalGordura":0,"totalAlcool":0,"totalKcal":0},"5": {"numero_refeicao": 5,"nome_refeicao": "Ceia","totalProteina": 0,"totalCarboidrato": 0,"totalGordura": 0,"totalAlcool": 0,"totalKcal": 0}}

entrada_usuario_base = f"Hoje é dia {dates['hoje']}. Um resumo geral de hoje: {refeicoes_usuario}. "

### Resposta Completa

In [14]:
with open(jsonl_file_path, "w", encoding="utf-8") as jsonl_file:
    for index, row in df.head(1000).iterrows():
        index_data = random.randint(0, 2)
        data_chave = datas_chaves[index_data]
        numero_refeicao = random.randint(1, 5)
        nome_refeicao = refeicoes_usuario[str(numero_refeicao)]['nome_refeicao']
        porcao_padrao = 1
        qtde_utilizada_porcao = random.choice(valores_quantidades)
        qtde_utilizada_gramas = qtde_utilizada_porcao * porcao_padrao
        fator_multiplicador = qtde_utilizada_gramas / row['porcao_padrao']
        # qtde_utilizada = round(qtde_utilizada_gramas / porcao_padrao, 1)
        entrada_usuario = entrada_usuario_base + f"Adicionar {qtde_utilizada_gramas} gramas de {row['nome_alimento']} no meu {nome_refeicao} de {data_chave}."
        arguments = {
            "alimentos": [{
                    "nome_consumo":row["nome_alimento"],
                    "numero_refeicao":numero_refeicao,
                    "id_alimento": None,
                    "id_prato":None,
                    "dt_dia":dates[data_chave],
                    "unidade_medida":"GRAMA",
                    "porcao_padrao": porcao_padrao,
                    "qtde_utilizada":qtde_utilizada_porcao,
                    "qtde_proteina":round(fator_multiplicador * row["qtde_proteina"], 1),
                    "qtde_carboidrato":round(
                        fator_multiplicador * row["qtde_carboidrato"], 1
                    ),
                    "qtde_gordura":round(fator_multiplicador * row["qtde_gordura"], 1),
                    "qtde_alcool":round(fator_multiplicador * row["qtde_alcool"], 1),
                    "kcal":round(fator_multiplicador * row["kcal"], 1),
                }]
         }

        resposta_assistente = {
            "id": "call_V80O8ygGntZ7zyzexd2G7Jfd",
            "type": "function",
            "function": {
                "name": "add_consumo_alimento",
                "arguments": json.dumps(arguments),
            },
        }

        mensagem = {
            "messages": [
                {
                    "role": "system",
                    "content": content,
                },
                {   "role": "user", 
                    "content": entrada_usuario
                },
                {
                    "role": "assistant",
                    "content": None,
                    "tool_calls": [resposta_assistente],
                },
            ]
        }

        jsonl_file.write(json.dumps(mensagem, ensure_ascii=False) + "\n")


print(f"Arquivo {jsonl_file_path} criado com sucesso.")

Arquivo output-backend.jsonl criado com sucesso.


### Sem refeição

In [15]:
with open(jsonl_file_path, 'a', encoding='utf-8') as jsonl_file:

    for index, row in df.head(100).iterrows():
        index_data = random.randint(0, 2)
        data_chave = datas_chaves[index_data]
        numero_refeicao = random.randint(1, 8)
        porcao_padrao = 1
        qtde_utilizada_porcao = random.choice(valores_quantidades)
        qtde_utilizada_gramas = qtde_utilizada_porcao * porcao_padrao
        fator_multiplicador = qtde_utilizada_gramas / row['porcao_padrao']
        # qtde_utilizada = round(qtde_utilizada_gramas / porcao_padrao, 1)
        entrada_usuario = entrada_usuario_base + f"Lembrando que hoje é dia {dates['hoje']}. Adicionar {qtde_utilizada_gramas} gramas de {row['nome_alimento']} {data_chave}."

        mensagem = {
            "messages": [
                {
                    "role": "system",
                    "content": content,
                },
                {
                    "role": "user",
                    "content": entrada_usuario
                },
                {
                    "role": "assistant",
                    "content": "Por favor informe a refeição.",
                    "tool_calls": None,
                },
            ]
        }
        
        jsonl_file.write(json.dumps(mensagem, ensure_ascii=False) + "\n")


print(f"Arquivo {jsonl_file_path} criado com sucesso.")

Arquivo output-backend.jsonl criado com sucesso.


### Sem alimento

In [16]:
with open(jsonl_file_path, 'a', encoding='utf-8') as jsonl_file:

    for index, row in df.head(50).iterrows():
        index_data = random.randint(0, 2)
        data_chave = datas_chaves[index_data]
        numero_refeicao = random.randint(1, 5)
        nome_refeicao = refeicoes_usuario[str(numero_refeicao)]['nome_refeicao']
        porcao_padrao = 1
        qtde_utilizada_porcao = random.choice(valores_quantidades)
        qtde_utilizada_gramas = qtde_utilizada_porcao * porcao_padrao
        fator_multiplicador = qtde_utilizada_gramas / row['porcao_padrao']
        # qtde_utilizada = round(qtde_utilizada_gramas / porcao_padrao, 1)
        entrada_usuario = entrada_usuario_base + f"Adicionar no meu {nome_refeicao} de {data_chave}."

        mensagem = {
            "messages": [
                {
                    "role": "system",
                    "content": content,
                },
                {
                    "role": "user",
                    "content": entrada_usuario
                },
                {
                    "role": "assistant",
                    "content": "Me informe o nome do alimento por favor.",
                    "tool_calls": None,
                },
            ]
        }
        
        jsonl_file.write(json.dumps(mensagem, ensure_ascii=False) + '\n')

print(f"Arquivo {jsonl_file_path} criado com sucesso.")

Arquivo output-backend.jsonl criado com sucesso.


### Sem Quantidade

In [17]:
with open(jsonl_file_path, 'a', encoding='utf-8') as jsonl_file:

    for index, row in df.head(100).iterrows():
        index_data = random.randint(0, 2)
        data_chave = datas_chaves[index_data]
        numero_refeicao = random.randint(1, 5)
        nome_refeicao = refeicoes_usuario[str(numero_refeicao)]['nome_refeicao']
        porcao_padrao = 1
        qtde_utilizada_porcao = random.choice(valores_quantidades)
        qtde_utilizada_gramas = qtde_utilizada_porcao * porcao_padrao
        fator_multiplicador = qtde_utilizada_gramas / row['porcao_padrao']
        # qtde_utilizada = round(qtde_utilizada_gramas / porcao_padrao, 1)
        entrada_usuario = entrada_usuario_base + f"Adicionar {row['nome_alimento']} na minha refeicao do {nome_refeicao} de {data_chave}."

        mensagem = {
            "messages": [
                {
                    "role": "system",
                    "content": content,
                },
                {
                    "role": "user",
                    "content": entrada_usuario
                },
                {
                    "role": "assistant",
                    "content": "Por favor informe a quantidade.",
                    "tool_calls": None,
                },
            ]
        }
        
        jsonl_file.write(json.dumps(mensagem, ensure_ascii=False) + '\n')

print(f"Arquivo {jsonl_file_path} criado com sucesso.")

Arquivo output-backend.jsonl criado com sucesso.


# Assumir data Atual sem infor

In [18]:
with open(jsonl_file_path, 'a', encoding='utf-8') as jsonl_file:
    
    for index, row in df.head(150).iterrows():
        index_data = random.randint(0, 2)
        data_chave = datas_chaves[index_data]
        numero_refeicao = random.randint(1, 5)
        porcao_padrao = 1
        qtde_utilizada_porcao = random.choice(valores_quantidades)
        qtde_utilizada_gramas = qtde_utilizada_porcao * porcao_padrao
        fator_multiplicador = qtde_utilizada_gramas / row['porcao_padrao']
        # qtde_utilizada = round(qtde_utilizada_gramas / porcao_padrao, 1)
        entrada_usuario = entrada_usuario_base + f"Adicionar {qtde_utilizada_gramas} gramas de {row['nome_alimento']} {row['estado_alimento']} na minha refeicao {numero_refeicao}."

        arguments = {
            "alimentos": [{
                    "numero_refeicao":numero_refeicao,
                    "id_alimento": None,
                    "id_prato":None,
                    "dt_dia":dates['hoje'],
                    "unidade_medida":"GRAMA",
                    "porcao_padrao": porcao_padrao,
                    "qtde_utilizada":qtde_utilizada_porcao,
                    "qtde_proteina":round(fator_multiplicador * row["qtde_proteina"], 1),
                    "qtde_carboidrato":round(
                        fator_multiplicador * row["qtde_carboidrato"], 1
                    ),
                    "qtde_gordura":round(fator_multiplicador * row["qtde_gordura"], 1),
                    "qtde_alcool":round(fator_multiplicador * row["qtde_alcool"], 1),
                    "kcal":round(fator_multiplicador * row["kcal"], 1),
                }]
        }

        resposta_assistente = {
            "id": "call_V80O8ygGntZ7zyzexd2G7Jfd",
            "type": "function",
            "function": {
                "name": "add_consumo_alimento",
                "arguments": json.dumps(arguments)
            },
        }

        mensagem = {
            "messages": [
                {
                    "role": "system",
                    "content": content,
                },
                {"role": "user", "content": entrada_usuario},
                {
                    "role": "assistant",
                    "content": None,
                    "tool_calls": [resposta_assistente],
                },
            ]
        }

        jsonl_file.write(json.dumps(mensagem, ensure_ascii=False) + "\n")


print(f"Arquivo {jsonl_file_path} criado com sucesso.")

Arquivo output-backend.jsonl criado com sucesso.
